### Most backed projects

v2 - scrap backer from the first page.

In [38]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser
import pickle
import sys
from mpi4py import MPI


def load_pickle(fname):
    f = open(fname)
    var = pickle.load(f)
    f.close()
    return var

def dump_pickle(variable,fname):
    f = open(fname, 'w')
    pickle.dump(variable, f)
    f.close()
    return None

def extract_backer_no_from_chunk(backer_chunk_n):
    dummy_backer_chunk = HTMLParser.HTMLParser().unescape(str(backer_chunk_n).decode('utf-8'))
    #print 'dummy_backer_chunk'
    #print dummy_backer_chunk
    backer_no = re.findall('and (.*?) backers', dummy_backer_chunk)[0].replace(",", "")
    #print 'backer_no'
    #print backer_no
    return backer_no

def filter_extracted_backer_no(backer_chunk_n):
    try:
        ans = int(backer_chunk_n)
    except:
        splitting = backer_chunk_n.split()
        ans = int(splitting[-1])
    return ans

pageno_array = np.arange(1,202,40)
# machine_no = 0
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name() # name of the process
print 'comm,rank,size,name =',comm,rank,size,name

machine_no = rank
for pageno in range(pageno_array[machine_no],pageno_array[machine_no+1]):
# for pageno in range(1,4):
# for pageno in range(2,pageno_array[machine_no+1]):

    print 'At the beginning of page '+str(pageno)
    
    proj_title_title = []
    proj_title_link = []
    proj_blurb_title = []
    number_of_backers = []
    dollar_sign_list = []
    amount_list = []
    proj_location_country = []
    Location = []
    currency = []
    proj_location_shortname = []
    proj_location_state = []
    proj_location_type = []
    goal_amount = []
    big_category_list = []
    small_category_list = []
    creater_list = []
    
    
    r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?woe_id=0&sort=most_backed&seed=2484532&page='+str(pageno)).read()
    soup = BeautifulSoup(r, 'html.parser')
#     print soup
#     sys.exit("Error message")
    
    #backer_chunk = soup.find_all("p",class_="project-profile-byline type-12")
    #print backer_chunk
    backer_chunk = soup.find_all("div", class_="project-profile-footer__content")
    #print backer_chunk
    
    
    number_of_backers_string = [extract_backer_no_from_chunk(xx) for xx in backer_chunk]
    number_of_backers = [filter_extracted_backer_no(xx) for xx in number_of_backers_string]
    if len(number_of_backers)<20:
        print 'Page #'+str(pageno)+' has an exception of live project.'
        continue
        
#     print 'number_of_backers'
#     print number_of_backers
    
    
    #<p class="project-profile-byline type-12">
#     print type(backer_chunk)
#     print backer_chunk[10]
    
    proj_chunk = soup.find_all("div", class_="project-card-content")
#     print proj_chunk[10]
#     sys.exit("Error message")
    
    
    
    for items in proj_chunk:
        # === Project descriptions ===
        title_str = HTMLParser.HTMLParser().unescape(str(items).decode('utf-8')) # remove the &quot
        tag = BeautifulSoup(title_str, 'html.parser')
        proj_title_title.append( tag.a.string )
        proj_title_link.append( tag.a['href'] )
        proj_blurb_title.append( tag.p.string.strip() )
        
#         print proj_blurb_title[-1]
        
        # === Next is to scrap things from each link! ===
        r2 = urllib.urlopen("https://www.kickstarter.com/"+proj_title_link[-1]).read()
        soup2 = BeautifulSoup(r2, 'html.parser')
        
#         print 'Find created by'
        creator_chunk = soup2.find_all("a",class_="hero__link remote_modal_dialog js-update-text-color")
        creater_list.append( creator_chunk[0].string.strip() )
#         sys.exit("Error message")
        
        
#         <a class="hero__link remote_modal_dialog js-update-text-color" data-modal-class="modal_project_by" data-modal-title="About the creator" href="/projects/mst3k/bringbackmst3k/creator_bio" style="color: #e7e7e7; ">
# Joel Hodgson
# </a>
        
        
        title_str2 = HTMLParser.HTMLParser().unescape(str(soup2).decode('utf-8')) # remove the &quot
    
        category_info = re.findall('"category":{(.*?)}', title_str2)
        category_info2 = re.findall('"slug":"(.*?)"', category_info[0])

        category_strings = category_info2[0].split('/')
        big_category_list.append(category_strings[0])
        if len(category_strings)>1:
            small_category_list.append(category_strings[1])
        else:
            small_category_list.append("N/A")
        
#         backers_info = re.findall('<b>(.*?) backers</b>', title_str2)
#         if len(backers_info)>0:
#             number_of_backers.append( int(backers_info[0].replace(",", "")) )
#         else:
#             try:
#                 backers_info = re.findall('data-backers-count="(.*?)"', title_str2)
#                 number_of_backers.append( int(backers_info[0].replace(",", "")) )
#             except:
#                 print 'error retrieving backers_info'
#                 print backers_info
#                 print proj_title_link[-1]
#                 print '-----------------'
            
        
        amount_list.append( re.findall('"pledged":(.*?),', title_str2)[0] )
        dollar_sign_list.append( re.findall('"currency":"(.*?)"', title_str2)[0] )
        
        goal_info = re.findall('pledged of <span class="money">(.*?)</span>', title_str2)
        pledged_amount_number = ''.join( re.findall('\d+', goal_info[0]) )
        
        goal_amount.append( pledged_amount_number )
        
        country_info = re.findall('"country":"(.*?)",', title_str2)
        proj_location_country.append(country_info[0])
        
        location_info = re.findall('"short_name":"(.*?)",', title_str2)
        proj_location_shortname.append( location_info[0] )
        
        state_info = proj_location_shortname[-1].split(',')
        proj_location_state.append( state_info[-1] )
        
        type_info = re.findall('"type":"(.*?)","is_root"', title_str2)
        proj_location_type.append( type_info[0] )

    try:
        df = pd.DataFrame({'Project title':proj_title_title, 'Project Link':proj_title_link,\
                           'Project blurb':proj_blurb_title, 'Creator':creater_list, \
                           'Category':big_category_list, 'Sub-category':small_category_list,\
                           'Location':proj_location_shortname, 'Country': proj_location_country,\
                          'State': proj_location_state, 'Type':proj_location_type, \
                           'number_of_backers':number_of_backers,\
                          'Dollar sign': dollar_sign_list, 'Amount':amount_list, 'Goal':goal_amount})
        display(df.head())
    except:
        print 'Page #'+str(pageno)+' has an exception.'
#         print len(proj_title_title)
#         print len(number_of_backers)
#         print dollar_sign_list
#         print amount_list
#         print goal_amount
        
        continue
    #     break

    if pageno==pageno_array[machine_no]:
        df2 = df
    else:
        df2 = pd.concat([df2,df])

    

df2.to_csv('most_backed_project_df2_'+str(machine_no)+'.csv',encoding='utf-8')

At the beginning of page 1


KeyboardInterrupt: 

In [37]:
# df3 = pd.read_csv('most_backed_project_df2_'+str(machine_no)+'.csv')
# df3

,Unnamed: 0,Amount,Category,Country,Creator,Dollar sign,Goal,Location,Project Link,Project blurb,Project title,State,Sub-category,Type,number_of_backers
0,0,5764229.38,film & video,US,\nJoel Hodgson\n,USD,2000000,"Minneapolis, MN",/projects/mst3k/bringbackmst3k?ref=most_backed,You did it: you brought back MYSTERY SCIENCE T...,Bring Back MYSTERY SCIENCE THEATER 3000,MN,television,Town,48270
1,1,3105473.10,film & video,US,\nZach Braff\n,USD,2000000,"Los Angeles, CA",/projects/1869987317/wish-i-was-here-1?ref=mos...,"""Hell, there are no rules here - we're trying ...",WISH I WAS HERE,CA,narrative film,Town,46520
2,2,9192055.66,design,US,\nBAUBAX LLC\n,USD,20000,"Chicago, IL",/projects/597538543/the-worlds-best-travel-jac...,"TRAVEL JACKET with built-in Neck Pillow, Eye M...",The World's Best TRAVEL JACKET with 15 Feature...,IL,product design,Town,44949
3,3,2229344.36,games,US,\nUber Entertainment Inc\n,USD,900000,"Kirkland, WA",/projects/659943965/planetary-annihilation-a-n...,Planetary Annihilation brings RTS gameplay to ...,Planetary Annihilation - A Next Generation RTS,WA,video games,Town,44162
4,4,2032434.63,games,US,\nLarian Studios LLC\n,USD,500000,"Sacramento, CA",/projects/larianstudios/divinity-original-sin-...,"An epic RPG with turn-based combat, cooperativ...",Divinity: Original Sin 2,CA,video games,Town,42713
5,5,2785537.13,games,US,\nHarebrained Schemes LLC\n,USD,250000,"Kirkland, WA",/projects/webeharebrained/battletech?ref=most_...,Missed the Kickstarter? Become a Late Reinforc...,BATTLETECH - Tactical 'Mech Combat Returns to ...,WA,video games,Town,41733
6,6,2071927.56,technology,US,\nNext Thing Co.\n,USD,50000,"Oakland, CA",/projects/1598272670/chip-the-worlds-first-9-c...,"Built for work, play, and everything in betwee...",CHIP - The World's First Nine Dollar Computer,CA,hardware,Town,39560
7,7,1022120.70,design,US,\nPolygons Design\n,USD,10000,"Wilmington, DE",/projects/stillalive/polygons-the-flat-4-in-1-...,Polygons is the origami-like measuring spoon t...,Polygons | The Flat 4-in-1 Measuring Spoon,DE,product design,Town,36863
8,8,1731465.98,technology,US,\nPINE64 Inc.\n,USD,31416,"San Francisco, CA",/projects/pine64/pine-a64-first-15-64-bit-sing...,PINE A64 is the world's first 64-bit expandabl...,"PINE A64, First $15 64-Bit Single Board Super ...",CA,hardware,Town,36781
9,9,1836447.15,games,US,\nHarebrained Schemes LLC\n,USD,400000,"Bellevue, WA",/projects/webeharebrained/shadowrun-returns?re...,Shadowrun Returns brings back one of our most ...,Shadowrun Returns,WA,video games,Town,36276


## Dollar-sign converter
http://utf8-chartable.de/unicode-utf8-table.pl?start=8320&number=128&names=-&utf8=string-literal

## Positive and Negative word List
Tutorial: http://nealcaren.web.unc.edu/pizza-twitter-and-apis

Positive List: http://www.unc.edu/~ncaren/haphazard/positive.txt        

Negative List: http://www.unc.edu/~ncaren/haphazard/negative.txt